In [2]:
import pandas as pd
import subprocess
import gradio as gr
import re

c:\Users\THINKPAD\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:

df = pd.read_csv("chatbot_products.csv")

# Function to search products and generate a prompt for TinyLlama
def generate_product_prompt(category=None, max_price=None, n_results=5):
    filtered = df.copy()

    if category:
        filtered = filtered[filtered['category'].str.contains(category, case=False)]

    if max_price:
        filtered = filtered[filtered['price'] <= max_price]

    top_products = filtered.head(n_results)

    if top_products.empty:
        return "Sorry, I couldn't find any products matching your request."

    prompt = "Here are some products you might like:\n\n"
    for _, row in top_products.iterrows():
        prompt += f"- {row['name']} ({row['category']}) - ${row['price']:.2f}\n  Description: {row['description']}\n\n"

    prompt += "Let me know if you want more options or help choosing!"
    return prompt


In [4]:
def chat_with_tinyllama(prompt):
    try:
        result = subprocess.run(
            ['ollama', 'run', 'tinyllama'],
            input=prompt.encode('utf-8'),
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            timeout=90 
        )
        response = result.stdout.decode('utf-8')
        return response
    except subprocess.TimeoutExpired:
        return "⚠️ Timeout: The model took too long to respond."
    except Exception as e:
        return f"❌ Error: {str(e)}"


In [5]:
# Get a filtered product list
prompt = generate_product_prompt(category="smartphone", max_price=400)

# Send it to TinyLlama
response = chat_with_tinyllama(prompt)

# Print the response
print("🤖 TinyLlama says:\n")
print(response)


🤖 TinyLlama says:

Sure, here are some additional products to consider:

- Honor Magicbook (Tablet) - $297.51
   Descripton: Majority get campaign political I on whose reduce because pattern organization live.

- Xiaomi Redmi Note 9 Pro (Smartphone) - $346.08
   Description: Suffeir in building dark kind side husband ten dec decade student left.

- Oppo A17 (Smartphone) - $250.51
   Descripton: Much get campaign political I on whose reduce because pattern organization live.

- Asus ZenFone 8 Flip (Phone) - $450.96
   Description: Majority laugh everyone seek who Democrat note nothing.

- OnePlus Nord (Smartphone) - $371.64
   Description: Get campaign political I on whose reduce because pattern organization live.

Let me know if you have any other questions or need more options.




In [6]:
def chatbot_response(user_input):
    
    prompt = generate_product_prompt(category="smartphone", max_price=400)
    full_prompt = prompt + f"\n\nUser asked: {user_input}\nAssistant:"
    return chat_with_tinyllama(full_prompt)

# Gradio app
demo = gr.Interface(
    fn=chatbot_response,
    inputs="text",
    outputs="text",
    title="🛍️ E-Commerce Chatbot (TinyLlama)",
    description="Ask about products, deals, or recommendations!"
)

demo.launch()


* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
